<a href="https://colab.research.google.com/github/raz0208/ModernBERT/blob/main/ModernBERT_TokenEmbedding_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extract embedding form inpot text using ModernBERT Version 1

In [1]:
# Install Neo4j
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.1 MB/s eta 0:00:00


In [2]:
# import required libraries
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from neo4j import GraphDatabase

### Load NLP and ModernBert models

In [3]:
# Load ModernBERT tokenizer and model from Hugging Face
MODEL_NAME = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

### Extract emmbedings based on full text

In [4]:
# Function to get inpout text and return full text embedding
def get_text_embedding(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings (use CLS token for sentence-level embedding)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, hidden_size]

    return cls_embedding.squeeze().numpy()

## Use Neo4j to connect the graph database

In [7]:
# Define Neo4j connection credentials
NEO4J_URI = "neo4j://143.225.233.156:7687"
NEO4J_USER = "rezaazari"
NEO4J_PASSWORD = "rAzari987"

# Initialize the driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Function to test connection
def test_connection():
    with driver.session() as session:
        greeting = session.run("RETURN 'Connected to Neo4j' AS message").single()["message"]
        print(greeting)

if __name__ == "__main__":
    test_connection()

Connected to Neo4j


In [8]:
# Function to run Cypher query
def run_query(cypher_query, parameters=None):
    with driver.session() as session:
        result = session.run(cypher_query, parameters or {})
        return [record.data() for record in result]

# Query of showing example nodes
query = "MATCH (n) RETURN n LIMIT 5"
results = run_query(query)
for r in results:
    print(r)

{'n': {'date': '1-12-1987', 'journal': 'The Journal of Cell Biology', 'hub': 0.0, 'auth': 2.6175247320960168e-12, 'subjects': 'Articles', 'pmc': 'PMC2114721', 'abstract': 'Meiosis I in males of the Dipteran Sciara coprophila results in the nonrandom distribution of maternally and paternally derived chromosome sets to the two division products. Based on an earlier study (Kubai, D.F. 1982. J. Cell Biol. 93:655-669), I suggested that the meiosis I spindle does not play a direct role in the nonrandom sorting of chromosomes but that, instead, haploid sets are already separated in prophase nuclei well before the onset of spindle formation. Here I report more direct evidence that this hypothesis is true; this evidence was gained from ultrastructural reconstruction analyses of the arrangement of chromosomes in germ line nuclei (prophase nuclei in spermatogonia and spermatocytes) of males heterozygous for an X- autosome chromosome translocation. Because of this translocation, the maternal and p

In [9]:
# Function to find similar nodes using cosine similarity
def find_similar_nodes(text_embedding, top_n):
     embedding_list = text_embedding.tolist()
     cypher_query = """
     MATCH (n)-[:HAS_EMBEDDING]->(e:ABSTRACT)
     WHERE e.embedding IS NOT NULL
     WITH n, labels(n) AS labels, gds.similarity.cosine($sent_embedding, e.embedding) AS similarity
     RETURN n, labels, similarity
     ORDER BY similarity DESC
     LIMIT $limit
     """
     parameters = {"sent_embedding": embedding_list, "limit": top_n}
     results = run_query(cypher_query, parameters)
     return results

### Exacute the app and get output

In [ ]:
### --- ### Sample text for test ### --- ###

# 1- This is an application about Breast Cancer.
# 2- Treating high blood pressure, high blood lipids, diabetes.
# 3- Heart failure, heart attack, stroke, aneurysm, peripheral artery disease, sudden cardiac arrest. Deaths: 17.9 million / 32% (2015)
# 4- Heart failure and stroke are common causes of death.

In [ ]:
# Example usage (Sentence: This is an application about Breast Cancer.)
if __name__ == "__main__":
    user_text = input("Enter your text: ")

    # Get sentence embedding
    full_text_embedding = get_text_embedding(user_text)
    print("\nSentence Embedding vector shape:", full_text_embedding.shape)
    print("Sentence Embedding (first 10 values):", full_text_embedding[:10])

    # Call function to run similarity query
    similar_nodes = find_similar_nodes(full_text_embedding, top_n=5)

    # Show the result
    print(f"\nTop {len(similar_nodes)} similar nodes:")
    for node_data in similar_nodes:
      print(f"Node: {node_data['n']}, Similarity: {node_data['similarity']:.4f}")

Enter your text: Heart failure and stroke are common causes of death.

Sentence Embedding vector shape: (768,)
Sentence Embedding (first 10 values): [ 0.29387805 -1.011311   -0.8573238  -0.0694401  -0.75960284 -0.7222283
 -1.1270415  -1.2861124   0.33987576 -0.52254   ]


### Import extracted data to local dataset

In [ ]:
# Define Neo4j connection credentials
# --- Connect to Second Database ---
SECOND_NEO4J_URI = "neo4j+s://17c6383a.databases.neo4j.io"
SECOND_NEO4J_USER = "neo4j"
SECOND_NEO4J_PASSWORD = "MMJrt6Cc0cp4VQn6QJJphqXFUUytZOpc0ip1tErl-4U"
second_driver = GraphDatabase.driver(SECOND_NEO4J_URI, auth=(SECOND_NEO4J_USER, SECOND_NEO4J_PASSWORD))

# Function to test connection
def test_connection():
    with second_driver.session() as session:
        greeting = session.run("RETURN 'Connected to Neo4j' AS message").single()["message"]
        print(greeting)

if __name__ == "__main__":
    test_connection()

In [ ]:
def import_nodes_to_second_db(nodes_data):
    with second_driver.session() as session:
        for record in nodes_data:
            node_props = record['n']
            labels = record['labels']  # From the modified query

            # Build Cypher query
            label_string = ":".join(labels)
            prop_keys = ", ".join(f"{k}: ${k}" for k in node_props.keys())
            cypher_query = f"CREATE (n:{label_string} {{ {prop_keys} }})"

            session.run(cypher_query, node_props)

    print(f"✅ Imported {len(nodes_data)} nodes to the second Neo4j database.")

In [ ]:
# # Import to second Neo4j
# import_nodes_to_second_db(similar_nodes)

In [ ]:
# Function to run Cypher query
def run_query(cypher_query, parameters=None):
    with second_driver.session() as session:
        result = session.run(cypher_query, parameters or {})
        return [record.data() for record in result]

# Query of showing example nodes
query = "MATCH (n) RETURN n LIMIT 5"
results = run_query(query)
for r in results:
    print(r)